In [1]:
!pip install ipython-sql
import sqlite3
import pyodbc


def connect(dbname):
    conn = sqlite3.connect(dbname)
    
    conn.execute("CREATE TABLE IF NOT EXISTS OYO_HOTELS (NAME TEXT, ADDRESS TEXT, AMENITIES TEXT)")
    print("table created successfully!")
    conn.close()
    
def insert_into_table(dbname, values):
    conn = sqlite3.connect(dbname)
    print("Inserted into table: "+ str(values))
    insert_sql = "INSERT INTO OYO_HOTELS (NAME, ADDRESS, AMENITIES) VALUES(?,?,?)"
    
    conn.execute(insert_sql, values)
    conn.commit()
    conn.close()
    
def get_hotel_info(dbname):
    conn = sqlite3.connect(dbname)
    
    cur = conn.cursor()
    
    cur.execute("SELECT * FROM OYO_HOTELS")
    
    table_data = cur.fetchall()
    
    for record in table_data:
        print(record)
        
    conn.close()

In [33]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import argparse

import mysql.connector

#import connect

parser = argparse.ArgumentParser()
parser.add_argument("--page_num_max", help="Enter the number of pages to parse", type = int)
parser.add_argument("--dbname", help="Enter the name of db", type=str)
args = parser.parse_args(args=[])


oyo_url = "https://www.oyorooms.com/hotels-in-bangalore/?page="
headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
page_num_MAX= args.page_num_max
scraped_into_list=[]
#connect.connect(args.dbname)

for page_num in range(1,10):
    url = oyo_url + str(page_num)
    print("GET Request for: "+url)
    #req = requests.get(oyo_url + str(page_num), headers = headers)
    req = requests.get(url)
    content= req.content
    #print(content)

    print("##############################################################################")

    soup= BeautifulSoup(content, "html.parser")
    #print(soup)
    
    all_hotels = soup.find_all("div", {"class": "hotelCardListing"})
    

    for hotel in all_hotels:
        hotel_dict={}
        hotel_dict["name"] = hotel.find("h3",{"itemprop": "name"}).text 
        hotel_dict["address"] = hotel.find("span", {"itemprop":"streetAddress"}).text
        #hotel_price = hotel.find("span",{"class": "listingPrice"}).text

        #try:
            # hotel_dict["rating"]= hotel.find("span", {"class": "hotelRating_ratingSummary"}).text
        #except AttributeError:
            #pass
    
        parent_amenities_element = hotel.find("div",{"class": "amenityWrapper"})
    
        amenities_list =[]
        for amenity in parent_amenities_element.find_all("div",{"class": "amenityWrapper_amenity"}):
               amenities_list.append(amenity.find("span",{"class": "d-body-sm"}).text.strip())
        hotel_dict["amenity"]=", ".join(amenities_list[:-1])
    
        scraped_into_list.append(hotel_dict)
        connect.insert_into_table(args.dbname, tuple(hotel_dict.values()))
        connect.get_hotel_info(args.dbname)
        #print(hotel_name, hotel_address ,  amenities_list )
    
dataFrame = pd.DataFrame(scraped_into_list)
print("Creating csv file...")
dataFrame.to_csv("Oyo.csv")
#connect.get_hotel_info(args.dbname)

GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=1
##############################################################################
GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=2
##############################################################################
GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=3
##############################################################################
GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=4
##############################################################################
GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=5
##############################################################################
GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=6
##############################################################################
GET Request for: https://www.oyorooms.com/hotels-in-bangalore/?page=7
####################################